In [46]:
import numpy as np

NETLIST_FILE_PATH = r'./testcases/newtc2.txt'
VSRC = 'Vsrc'
ISRC = 'Isrc'
RESISTOR = 'R'
INDUCTOR = 'I'
CAPACITOR = 'C'
GND = 'V0'
EOF = '-1'

In [47]:
class Component:
    def __init__(self, comp_type, node1, node2, value, init_value):
        self.comp_type = comp_type
        self.node1 = node1
        self.node2 = node2
        self.value = float(value)
        self.init_value = float(init_value)
    
    def __str__(self):
        return f'{self.comp_type} {self.node1} {self.node2} {self.value} {self.init_value}'

In [48]:
class Circuit:
    def __init__(self):
        self.circuit = []
        self.circuit_map = {}

        # utility
        self.naming_map = {}
        self.node_number = 0

        self.x_mat = None

    def __str__(self):
        string = ''
        for comp in self.circuit:
            string = string + f'{comp.node1} {comp.node2}\n' 
        return string

    # utility
    def find_node_number(self, node_name):
        for key, val in self.naming_map.items():
            if val == node_name: return key
            
    # utility
    def find_node_name(self, node_number):
        return self.naming_map[node_number]

    def get_nodes_count(self):
        return len(self.circuit_map)
    
    def get_voltage_source_count(self):
        return len([c for c in self.circuit if c.comp_type == VSRC or c.comp_type == INDUCTOR])

    def fill_x_mat(self):
        self.x_mat = np.zeros((self.get_nodes_count() + self.get_voltage_source_count(), 1))
    
    def update_x_mat(self, new_x_mat):
        self.x_mat = new_x_mat.copy()
    
    def add_component(self, component):
        self.circuit.append(component)
        
        if component.node1 != GND:
            if self.circuit_map.get(component.node1, None):
                self.circuit_map[component.node1].append(component)
            else:
                self.circuit_map[component.node1] = [component]
                self.naming_map[self.node_number] = component.node1
                self.node_number += 1

        if component.node2 != GND:
            if self.circuit_map.get(component.node2, None):
                self.circuit_map[component.node2].append(component)
            else:
                self.circuit_map[component.node2] = [component]
                self.naming_map[self.node_number] = component.node2
                self.node_number += 1
        
    def get_g_mat(self, time_step):
        g_mat = np.zeros((self.get_nodes_count(), self.get_nodes_count()))
        for i in range(g_mat.shape[0]):
            for j in range(g_mat.shape[1]):
                if i == j:
                    g_mat[i][j] = sum([1 / comp.value for comp in self.circuit_map[self.find_node_name(i)] if comp.comp_type == RESISTOR])
                    g_mat[i][j] += sum([comp.value / time_step for comp in self.circuit_map[self.find_node_name(i)] if comp.comp_type == CAPACITOR])
                else:
                    g_mat[i][j] = -sum([
                        1 / comp.value 
                        for comp in self.circuit_map[self.find_node_name(i)] 
                        if comp.comp_type == RESISTOR and (comp.node2 == self.find_node_name(j) or comp.node1 == self.find_node_name(j))
                    ])

                    g_mat[i][j] += -sum([
                        comp.value / time_step
                        for comp in self.circuit_map[self.find_node_name(i)] 
                        if comp.comp_type == CAPACITOR and (comp.node2 == self.find_node_name(j) or comp.node1 == self.find_node_name(j))
                    ])
        return g_mat
    
    def get_b_mat(self):
        b_mat = np.zeros((self.get_nodes_count(), self.get_voltage_source_count()))

        vsrc_index = 0
        for comp in self.circuit:
            if comp.comp_type == VSRC or comp.comp_type == INDUCTOR:
                if comp.node1 != GND:
                    b_mat[self.find_node_number(comp.node1), vsrc_index] = 1
                if comp.node2 != GND:
                    b_mat[self.find_node_number(comp.node2), vsrc_index] = -1
                vsrc_index += 1
        return b_mat


    def get_c_mat(self): 
        return np.transpose(self.get_b_mat())

    def get_d_mat(self, time_step): 
        d_mat = np.zeros((self.get_voltage_source_count(), self.get_voltage_source_count()))
        index = 0
        for comp in self.circuit:
            if comp.comp_type == VSRC:
                d_mat[index, index] = 0
                index += 1
            elif comp.comp_type == INDUCTOR:
                d_mat[index, index] = -comp.value / time_step
                index += 1
        return d_mat

    def get_i_mat(self, time_step): 
        i_mat = np.zeros((self.get_nodes_count(), 1))
        for comp in self.circuit:
            if comp.comp_type == ISRC:
                if comp.node1 != GND:
                    i_mat[self.find_node_number(comp.node1), 0] += comp.value
                if comp.node2 != GND:
                    i_mat[self.find_node_number(comp.node2), 0] -= comp.value
            
            if comp.comp_type == CAPACITOR:
                if comp.node1 != GND:
                    i_mat[self.find_node_number(comp.node1), 0] += (comp.value / time_step) * (self.x_mat[self.find_node_number(comp.node1)] - (0 if comp.node2 == GND else self.x_mat[self.find_node_number(comp.node2)]))
                if comp.node2 != GND:
                    i_mat[self.find_node_number(comp.node2), 0] -= (comp.value / time_step) * (self.x_mat[self.find_node_number(comp.node1)] - (0 if comp.node2 == GND else self.x_mat[self.find_node_number(comp.node2)]))
        return i_mat
    
    def get_e_mat(self, time_step): 
        e_mat = np.zeros((self.get_voltage_source_count(), 1))
        index = 0
        for comp in self.circuit:
            if comp.comp_type == VSRC:
                e_mat[index, 0] = comp.value
                index += 1
            elif comp.comp_type == INDUCTOR:
                e_mat[index, 0] = - (comp.value / time_step) * self.x_mat[max(self.naming_map) + index + 1]
                index += 1
        return e_mat


In [49]:
class Simulation:
    def __init__(self, netlist_file_path):
        file = open(netlist_file_path)

        self.time_step = float(file.readline())
        self.iterations = int(file.readline())
        self.circuit = Circuit()

        for comp in file:
            if comp == EOF: break
            comp = comp.split(' ')[ : -1] + [comp.split(' ')[-1].strip('\n')]
            component = Component(*comp)
            self.circuit.add_component(component)

        file.close()
        self.circuit.fill_x_mat()
    
    def __str__(self):
        return f'{self.time_step} {self.iterations}\n{self.circuit}'
    
    def solve_one_iter(self):
        n = self.circuit.get_nodes_count()
        m = self.circuit.get_voltage_source_count()

        g_mat = self.circuit.get_g_mat(self.time_step)
        b_mat = self.circuit.get_b_mat()
        c_mat = self.circuit.get_c_mat()
        d_mat = self.circuit.get_d_mat(self.time_step)
     

        a_mat = np.zeros((n + m, n + m))
        a_mat[ : n, : n] = g_mat
        a_mat[ : n, n : n + m] = b_mat
        a_mat[n : n + m, : n] = c_mat
        a_mat[n : n + m, n : n + m] = d_mat

        i_mat = self.circuit.get_i_mat(self.time_step)
        e_mat = self.circuit.get_e_mat(self.time_step)

        z_mat = np.zeros((n + m, 1))
        z_mat[ : n, : ] = i_mat
        z_mat[n : n + m, :] = e_mat

        x_mat = np.matmul(np.linalg.inv(a_mat), z_mat)
        
        return np.round(x_mat, 10)  


    def solve(self):
        time = 0
        x_mat = None
        for _ in range(self.iterations):
            time += self.time_step
            x_mat = self.solve_one_iter()
            print(f'{time} | {x_mat}\n')
            self.circuit.update_x_mat(x_mat)
        return x_mat


   

In [50]:
simulation = Simulation(NETLIST_FILE_PATH)
x_mat = simulation.solve()
print()
print(x_mat)

0.1 | [[  0.51181102]
 [  6.14173228]
 [ -3.85826772]
 [  0.61417323]
 [-24.24409449]
 [ -0.38582677]]

0.2 | [[  0.94612189]
 [  6.23535247]
 [ -3.76464753]
 [  1.23770848]
 [-24.52693905]
 [ -0.76229152]]

0.30000000000000004 | [[  1.31256948]
 [  6.28961478]
 [ -3.71038522]
 [  1.86666995]
 [-24.84371526]
 [ -1.13333005]]

0.4 | [[  1.61981057]
 [  6.31203212]
 [ -3.68796788]
 [  2.49787317]
 [-25.19009471]
 [ -1.50212683]]

0.5 | [[  1.87559395]
 [  6.30902164]
 [ -3.69097836]
 [  3.12877533]
 [-25.56220303]
 [ -1.87122467]]


[[  1.87559395]
 [  6.30902164]
 [ -3.69097836]
 [  3.12877533]
 [-25.56220303]
 [ -1.87122467]]
